In [1]:
! pip install evaluate
! pip install sentence-transformers
! pip install SentencePiece
! pip install bert_score
! pip install --upgrade nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 16.2 MB/s eta 0:00:00
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.8.1 which is incompatible.


In [2]:
import csv
import json
import torch
import logging
import numpy as np 
import pandas as pd 
import evaluate
from evaluate import load


2024-03-29 21:56:13.712399: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-29 21:56:13.712542: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-29 21:56:13.857391: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
from transformers import T5Tokenizer,T5ForConditionalGeneration
import json

In [4]:
device=""
if torch.cuda.is_available():
    device = torch.device("cuda")
    logging.info(f'Using GPU: {torch.cuda.get_device_name()}')
    print(f'Using GPU: {torch.cuda.get_device_name()}')
else:
    device = torch.device("cpu")
    logging.info('Using CPU')
    print('Using CPU')

Using CPU


In [5]:
bertscore = load("bertscore")
meteor = evaluate.load("meteor")
bleu = evaluate.load("bleu")

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


In [6]:
tokenizer = T5Tokenizer.from_pretrained("google-t5/t5-small")
model = T5ForConditionalGeneration.from_pretrained("google-t5/t5-small").to(device)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [7]:
file_path = "/kaggle/input/assignment-3/data/Task2/"

In [8]:
testing_data = []
validation_data = []

In [9]:
# load json
with open(file_path + "test.json") as f:
    testing_data = json.load(f)
    testing_data = testing_data["translation"]

with open(file_path + "validation_data.json") as f:
    validation_data = json.load(f)
    validation_data = validation_data["translation"]


en_test_data = []
en_val_data = []

de_test_data = []
de_val_data = []

for i in range(len(testing_data)):
    en_test_data.append(testing_data[i]["en"])
    de_test_data.append(testing_data[i]["de"])
for i in range(len(validation_data)):
    de_val_data.append(validation_data[i]["de"])
    en_val_data.append(validation_data[i]["en"])




In [10]:

# en_test_data=en_test_data[:10]
# de_test_data=de_test_data[:10]
# en_val_data=en_val_data[:10]
# de_val_data=de_val_data[:10]



In [11]:
len(en_test_data),len(de_test_data),len(en_val_data),len(de_val_data)

(2999, 2999, 2169, 2169)

In [12]:
prefix = "translate English to German: "
test_model_input=[]
val_model_input=[]
for i in range(len(en_test_data)):
    test_model_input.append(prefix + en_test_data[i])
for i in range(len(en_val_data)):
    val_model_input.append(prefix + en_val_data[i])



# Testing Set

In [13]:
test_output = []

j=0;
for i in test_model_input:
    input_ids = tokenizer(i, return_tensors="pt").input_ids.to(device)
    output = model.generate(input_ids, max_length=256, num_beams=5, early_stopping=True)
    decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
    test_output.append(decoded_output)
    j+=1
    if( j%100 == 0):
        print(j)

print("Number of translations:", len(test_output))



100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
Number of translations: 2999


In [14]:
translated_csv_file_path = 'translated_wmt16_test_2B.csv'

# Write translated outputs to CSV
with open(translated_csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['en', 'de', 'translated_en']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    for en, de, translated_en in zip(en_test_data, de_test_data, test_output):
        writer.writerow({'en': en, 'de': de, 'translated_en': translated_en})

print(f"Translated CSV file saved successfully at: {translated_csv_file_path}")

Translated CSV file saved successfully at: translated_wmt16_test_2B.csv


In [15]:
def calculate_bleu(bp,precisions):
    weights_1 = np.array([1,0,0,0])
    weights_2 = np.array([0.5,0.5,0,0])
    weights_3 = np.array([0.33,0.33,0.33,0])
    weights_4 = np.array([0.25,0.25,0.25,0.25])
    logp = np.log(precisions)
        
    res1 = bp*np.exp(np.dot(logp,weights_1))
    res2 = bp*np.exp(np.dot(logp,weights_2))
    res3 = bp*np.exp(np.dot(logp,weights_3))
    res4 = bp*np.exp(np.dot(logp,weights_4))
    
    return [res1,res2,res3,res4]
    

In [16]:
results = bleu.compute(predictions=test_output, references=de_test_data)
bleu_score = calculate_bleu(results["brevity_penalty"],results["precisions"])
print(results)
print(f"Bleu-1 = {bleu_score[0]}")
print(f"Bleu-2 = {bleu_score[1]}")
print(f"Bleu-3 = {bleu_score[2]}")
print(f"Bleu-4 = {bleu_score[3]}")

{'bleu': 0.32736540896921046, 'precisions': [0.6344154908147412, 0.3875298630505737, 0.2609897411364482, 0.1816701906704339], 'brevity_penalty': 0.9962911493683022, 'length_ratio': 0.996298010180472, 'translation_length': 62437, 'reference_length': 62669}
Bleu-1 = 0.6320625385208741
Bleu-2 = 0.4939986352085154
Bleu-3 = 0.40252796811721286
Bleu-4 = 0.32736540896921046


In [17]:
# meteor
results = meteor.compute(predictions=test_output, references=de_test_data)
print(f"Meteor Score : {results}")

Meteor Score : {'meteor': 0.5900756151197717}


In [18]:
results = bertscore.compute(predictions=test_output, references=de_test_data, lang="de")
print("Bert-Score : ")
print(np.average(results["f1"]))
print(np.average(results["precision"]))
print(np.average(results["recall"]))

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Bert-Score : 
0.8761486683181223
0.8782275977552871
0.8744261644211718


# Validation Set

In [19]:
val_output = []

j=0;
for i in val_model_input:
    input_ids = tokenizer(i, return_tensors="pt").input_ids.to(device)
    output = model.generate(input_ids, max_length=256, num_beams=5, early_stopping=True)
    decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
    val_output.append(decoded_output)
    j+=1
    if( j%100 == 0):
        print(j)

len(val_output)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100


2169

In [20]:
translated_csv_file_path = 'translated_wmt16_val_2B.csv'

# Write translated outputs to CSV
with open(translated_csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['en', 'de', 'translated_en']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    for en, de, translated_en in zip(en_test_data, de_test_data, val_output):
        writer.writerow({'en': en, 'de': de, 'translated_en': translated_en})

print(f"Translated CSV file saved successfully at: {translated_csv_file_path}")

Translated CSV file saved successfully at: translated_wmt16_val_2B.csv


In [21]:
results = bleu.compute(predictions=val_output, references=de_val_data)
bleu_score = calculate_bleu(results["brevity_penalty"],results["precisions"])
print(results)
print(f"Bleu-1 = {bleu_score[0]}")
print(f"Bleu-2 = {bleu_score[1]}")
print(f"Bleu-3 = {bleu_score[2]}")
print(f"Bleu-4 = {bleu_score[3]}")

{'bleu': 0.28141189111781467, 'precisions': [0.5934127072437966, 0.3363887200112239, 0.21618226600985221, 0.1453287197231834], 'brevity_penalty': 1.0, 'length_ratio': 1.0152507907817443, 'translation_length': 44935, 'reference_length': 44260}
Bleu-1 = 0.5934127072437966
Bleu-2 = 0.4467855649281161
Bleu-3 = 0.35445087212474885
Bleu-4 = 0.28141189111781467


In [22]:
results = meteor.compute(predictions=val_output, references=de_val_data)
print("Meteor Score : ",results)

Meteor Score :  {'meteor': 0.5506565724113192}


In [23]:
results = bertscore.compute(predictions=val_output, references=de_val_data, lang="de")
print("Bert-Score : ")
print(np.average(results["f1"]))
print(np.average(results["precision"]))
print(np.average(results["recall"]))


Bert-Score : 
0.8647587318033488
0.8655421310660475
0.8643602566435459
